# * VINSIGHT : Data Monitoring
    Process required -> "ETL-AGG_PERF_NEWCO_SNAP"

## Parameter

In [125]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2024, 6, 17)

## Create DataFrame
    DB -> CSV file

In [207]:

# Input parameter
op_dir = 'data'
op_file = f'agg_perf_newco_snap_{str_curr_dt}'

# Connect : AKPIPRD
dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
conn = oracledb.connect(dsn)
print(f'\n{AKPIPRD_db} : Connected\n\nProcessing...')
cur = conn.cursor()

query = """
    -->> Actual Monthly

    SELECT TM_KEY_MTH, CENTER, METRIC_GRP, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, CHANNEL_CD, AGG_TYPE, UOM
    
        , CAST(SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_TMP END) AS DECIMAL(18,2)) AS P_ACTUAL
        , CAST(SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_TMP END) AS DECIMAL(18,2)) AS G_ACTUAL
        , CAST(SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_TMP END) AS DECIMAL(18,2)) AS H_ACTUAL
        , CAST(SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_TMP END) AS DECIMAL(18,2)) AS HH_ACTUAL

        , CAST(SUM(CASE WHEN AREA_TYPE = 'P' THEN TARGET_TMP END) AS DECIMAL(18,2)) AS P_TARGET
        , CAST(SUM(CASE WHEN AREA_TYPE = 'G' THEN TARGET_TMP END) AS DECIMAL(18,2)) AS G_TARGET
        , CAST(SUM(CASE WHEN AREA_TYPE = 'H' THEN TARGET_TMP END) AS DECIMAL(18,2)) AS H_TARGET
        , CAST(SUM(CASE WHEN AREA_TYPE = 'HH' THEN TARGET_TMP END) AS DECIMAL(18,2)) AS HH_TARGET

        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF, MIN(TM_KEY_DAY) MIN_DAY, MAX(TM_KEY_DAY) MAX_DAY, MAX(PPN_TM) PPN_TM, MAX(LOAD_DATE) LOAD_DATE
        
    FROM (
        SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY
            , CENTER, PRODUCT_GRP, COMP_CD, METRIC_GRP, METRIC_CD, METRIC_NAME, SEQ, ACTUAL_AS_OF, AGG_TYPE, RR_IND, GRY_IND, UOM, AREA_TYPE, AREA_CD, AREA_NAME
            , CASE 	WHEN AGG_TYPE = 'S' THEN ACTUAL_SNAP 
                    ELSE (CASE WHEN TM_KEY_DAY = MAX(TM_KEY_DAY) OVER(PARTITION BY METRIC_CD, TM_KEY_MTH) THEN ACTUAL_AGG END)
                    END ACTUAL_TMP
            , CASE 	WHEN AGG_TYPE = 'S' THEN TARGET_SNAP 
                    ELSE (CASE WHEN TM_KEY_DAY = MAX(TM_KEY_DAY) OVER(PARTITION BY METRIC_CD, TM_KEY_MTH) THEN TARGET_AGG END)
                    END TARGET_TMP
            , ACTUAL_SNAP, TARGET_SNAP, BASELINE_SNAP, ACTUAL_AGG, TARGET_AGG, BASELINE_AGG, PPN_TM, LOAD_DATE
            , CASE WHEN REGEXP_LIKE(METRIC_CD, '[0-9]A[A-K]$') THEN SUBSTR(METRIC_CD,-2) ELSE 'ALL' END CHANNEL_CD
        FROM AUTOKPI.AGG_PERF_NEWCO_SNAP NOLOCK
    ) TMP_MTH

    --WHERE CHANNEL_CD = 'ALL'
    GROUP BY TM_KEY_MTH, CENTER, METRIC_GRP, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, CHANNEL_CD, AGG_TYPE, UOM
    ORDER BY TM_KEY_MTH, CENTER, METRIC_GRP, PRODUCT_GRP, COMP_CD, METRIC_CD
"""


try:
    # Create Dataframe
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')
    cur.execute(query)
    rows = cur.fetchall()
    df = pd.DataFrame.from_records(rows, columns=[x[0] for x in cur.description])
    print(f'\n   -> DataFrame : {df.shape[0]} rows, {df.shape[1]} columns')

    # Generate CSV file
    df.to_csv(f'{op_dir}/{op_file}.csv', index=False, encoding='utf-8')
    print(f'\n   -> Generate "{op_file}.csv" successfully')

    # # Generate Excel file
    # df.to_excel(f'{op_dir}/{op_file}.xlsx', sheet_name='Data', index=False)
    # print(f'\n   -> Generate "{op_file}.xlsx" successfully')


except oracledb.DatabaseError as e:
    print(f'Error with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')



AKPIPRD : Connected

Processing...

   -> Execute query... 2024-06-17, 17:12:38

   -> DataFrame : 7198 rows, 23 columns

   -> Generate "agg_perf_newco_snap.csv" successfully

AKPIPRD : Disconnected


In [254]:
df.tail(1)

,TM_KEY_MTH,CENTER,METRIC_GRP,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,CHANNEL_CD,AGG_TYPE,UOM,...,HH_ACTUAL,P_TARGET,G_TARGET,H_TARGET,HH_TARGET,ACTUAL_AS_OF,MIN_DAY,MAX_DAY,PPN_TM,LOAD_DATE
7197,202406,Sales,Subs,TOL,TRUE,TB3S000700GEO,TOL %NAD 30DPDB2 (Due Date),ALL,N,%,...,NaN,NaN,NaN,NaN,NaN,NaN,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071


## Reconcile

In [255]:
''' Rawdata '''
data_src = f'{op_dir}/{op_file}.csv'
raw_df = pd.read_csv(data_src, low_memory=False)


''' Filters '''
# raw_df = raw_df[raw_df['TM_KEY_MTH'] >= 202401] # MONTH filter
# print(f'\nraw_df : {raw_df.shape[0]} rows, {raw_df.shape[1]} columns')
raw_df.tail()

,TM_KEY_MTH,CENTER,METRIC_GRP,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,CHANNEL_CD,AGG_TYPE,UOM,...,HH_ACTUAL,P_TARGET,G_TARGET,H_TARGET,HH_TARGET,ACTUAL_AS_OF,MIN_DAY,MAX_DAY,PPN_TM,LOAD_DATE
7193,202406,Sales,Sales,TVS,TRUE,TB4R001000,TVS Inflow M1,ALL,S,baht,...,4320.0,NaN,NaN,NaN,NaN,20240614.0,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071
7194,202406,Sales,Sales,TVS,TRUE,TB4R001000CORP,TVS Inflow M1,ALL,S,baht,...,4320.0,NaN,NaN,NaN,NaN,20240614.0,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071
7195,202406,Sales,Sales,TVS,TRUE,TB4R001600,TVS GA ARPU,ALL,N,baht,...,NaN,NaN,NaN,NaN,NaN,20240615.0,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071
7196,202406,Sales,Subs,Postpaid,TRUE,TB2S000700GEO,Postpaid %NAD 30DPDB2 (Due Date) : TMH,ALL,N,%,...,NaN,NaN,NaN,NaN,NaN,NaN,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071
7197,202406,Sales,Subs,TOL,TRUE,TB3S000700GEO,TOL %NAD 30DPDB2 (Due Date),ALL,N,%,...,NaN,NaN,NaN,NaN,NaN,NaN,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071


In [275]:
''' Generate Temp files '''

''' GROUP list '''
# grp_list_df = raw_df[['METRIC_GRP', 'PRODUCT_GRP']].drop_duplicates().reset_index(drop=True)
# grp_list_df.dropna(how='all')
# # grp_list_df.dropna(axis=1, how='all')
# # grp_list_df.dropna(subset=['PRODUCT_GRP'])
grp_list_df = raw_df.groupby(['METRIC_GRP', 'PRODUCT_GRP']).agg({'METRIC_CD': 'nunique', 'MIN_DAY': 'min', 'MAX_DAY': 'max'}).reset_index()
grp_list_df.rename(columns={'METRIC_CD': 'CNT_METRIC'}, inplace=True)
grp_list_df.to_excel(f'temp/Metric_Grp_List_{str_curr_dt}.xlsx', sheet_name='Data', index=False)
print(f'\n   -> Generate "Metric_Grp_List_{str_curr_dt}.xlsx" successfully')


''' METRIC list '''
# metric_list_df = raw_df[['METRIC_GRP', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']].drop_duplicates().reset_index(drop=True)
# metric_list_df.dropna(how='all')
tmp_df = raw_df
tmp_df['TMP_CD'] = tmp_df['METRIC_CD'].replace(r'AA$|AB$|AC$|AD$|AE$|AF$|AG$|AH$|AI$|AJ$|AK$', '', regex=True)
tmp_df['TMP_NAME'] = tmp_df['METRIC_NAME'].replace(r' : Account Executive| : B2B| : Branded Retail| : Contact Center| : Direct Sales| : Key Account| : Modern Trade| : Others| : Own Digital| : Retail Sales| : Wholesales', '', regex=True)
# tmp_df
metric_list_df = tmp_df.groupby(['METRIC_GRP', 'PRODUCT_GRP', 'COMP_CD', 'TMP_CD', 'TMP_NAME', 'AGG_TYPE', 'UOM'])\
    .agg({'CHANNEL_CD': 'nunique', 'ACTUAL_AS_OF': 'max', 'MIN_DAY': 'min', 'MAX_DAY': 'max'}).reset_index()
metric_list_df.rename(columns={'CHANNEL_CD': 'CNT_CHANNEL', 'ACTUAL_AS_OF': 'LAST_ACTUAL'}, inplace=True)
metric_list_df.to_excel(f'temp/Metric_Cd_List_{str_curr_dt}.xlsx', sheet_name='Data', index=False)
print(f'\n   -> Generate "Metric_Cd_List_{str_curr_dt}.xlsx" successfully')



   -> Generate "Metric_Grp_List_20240617.xlsx" successfully

   -> Generate "Metric_Cd_List_20240617.xlsx" successfully


In [279]:
''' Create Reconcile Data '''

rec_df = raw_df

''' Filters '''
rec_df = rec_df[rec_df['TM_KEY_MTH']==202406]
rec_df = rec_df[rec_df['CHANNEL_CD']=='ALL']

# my_str = 'Prepaid Topping|Prepaid Pay per Use'
# my_str = '^Postpaid Revenue.*DTAC$'
# my_str = 'Prepaid Inflow M1'

# rec_df = rec_df[rec_df['METRIC_NAME'].str.contains(my_str)]
rec_df = rec_df.reset_index(drop=True)
# rec_df.tail(3)
rec_df

,TM_KEY_MTH,CENTER,METRIC_GRP,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,CHANNEL_CD,AGG_TYPE,UOM,...,G_TARGET,H_TARGET,HH_TARGET,ACTUAL_AS_OF,MIN_DAY,MAX_DAY,PPN_TM,LOAD_DATE,TMP_CD,TMP_NAME
0,202406,Profitability,Digital Growth Champion,TDG,TRUE,CAP00008,Digital %Gross Profit,ALL,N,%,...,NaN,NaN,NaN,20240430.0,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071,CAP00008,Digital %Gross Profit
1,202406,Profitability,Finance,Corporate,ALL,CAP00001,EBITDA %Ach,ALL,N,%,...,NaN,NaN,NaN,20240521.0,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071,CAP00001,EBITDA %Ach
2,202406,Profitability,Finance,Corporate,ALL,CAP00006,%EBITDA,ALL,N,%,...,NaN,NaN,NaN,20240430.0,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071,CAP00006,%EBITDA
3,202406,Profitability,Finance,Corporate,ALL,CAP00009,EBITDA,ALL,N,baht,...,NaN,NaN,NaN,20240430.0,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071,CAP00009,EBITDA
4,202406,Profitability,Finance,Corporate,TRUE,TCAP00001,EBITDA : TRUE,ALL,N,baht,...,NaN,NaN,NaN,20240430.0,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071,TCAP00001,EBITDA : TRUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,202406,Sales,Sales,TVS,TRUE,TB4R001000,TVS Inflow M1,ALL,S,baht,...,NaN,NaN,NaN,20240614.0,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071,TB4R001000,TVS Inflow M1
478,202406,Sales,Sales,TVS,TRUE,TB4R001000CORP,TVS Inflow M1,ALL,S,baht,...,NaN,NaN,NaN,20240614.0,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071,TB4R001000CORP,TVS Inflow M1
479,202406,Sales,Sales,TVS,TRUE,TB4R001600,TVS GA ARPU,ALL,N,baht,...,NaN,NaN,NaN,20240615.0,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071,TB4R001600,TVS GA ARPU
480,202406,Sales,Subs,Postpaid,TRUE,TB2S000700GEO,Postpaid %NAD 30DPDB2 (Due Date) : TMH,ALL,N,%,...,NaN,NaN,NaN,NaN,20240601,20240616,2024-06-17 11:10:20,2024-06-17 14:01:32.232071,TB2S000700GEO,Postpaid %NAD 30DPDB2 (Due Date) : TMH


In [282]:
''' Group by '''

agg_df = rec_df.groupby(['COMP_CD', 'METRIC_CD', 'METRIC_NAME', 'CHANNEL_CD', 'UOM']).agg({'P_ACTUAL': 'sum', 'P_TARGET': 'sum', 'LOAD_DATE': 'max'}).reset_index()
    #.sort_values(['COMP_CD', 'METRIC_CD']).reset_index(drop=True)

# df['DecimalCol'] = df['DecimalCol'].apply(lambda x: round(x, 2))
# agg_df[['P_ACTUAL', 'P_TARGET']] = agg_df[['P_ACTUAL', 'P_TARGET']].apply(lambda x: round(x, 2))
# agg_df = agg_df.astype({'P_ACTUAL': int, 'P_TARGET': int, 'LOAD_DATE': 'datetime64[ns]'})

# Create Temp File
# agg_df.to_excel(f'temp/Temp.xlsx', sheet_name='Data', index=False)
# print(f'\n -> Generate "Temp.xlsx" successfully')

agg_df

,COMP_CD,METRIC_CD,METRIC_NAME,CHANNEL_CD,UOM,P_ACTUAL,P_TARGET,LOAD_DATE
0,ALL,B0R000100,Total Revenue,ALL,baht,5.937636e+09,5.952347e+09,2024-06-17 14:01:32.232071
1,ALL,B0R00010001,Total Inflow M1,ALL,baht,2.016992e+08,0.000000e+00,2024-06-17 14:01:32.232071
2,ALL,B0R00010001CORP,Total Inflow M1,ALL,baht,2.016992e+08,0.000000e+00,2024-06-17 14:01:32.232071
3,ALL,B0R00010002,Total Gross Adds,ALL,subs,1.255615e+06,1.564200e+06,2024-06-17 14:01:32.232071
4,ALL,B0R0001002,%Revenue Growth (YTD YoY),ALL,%,5.670000e+00,4.700000e+00,2024-06-17 14:01:32.232071
...,...,...,...,...,...,...,...,...
475,TRUE,VIN00042,Revenue (Corporate),ALL,score,3.980000e+00,4.000000e+00,2024-06-17 14:01:32.232071
476,TRUE,VIN00043,Profitability (Corporate),ALL,score,0.000000e+00,4.000000e+00,2024-06-17 14:01:32.232071
477,TRUE,VIN00050,Mobile Subs Share (Subs) : TMH,ALL,subs,3.705530e+06,0.000000e+00,2024-06-17 14:01:32.232071
478,TRUE,VIN00061,Prepaid Subs Share (Subs) : TMH,ALL,subs,2.463159e+06,0.000000e+00,2024-06-17 14:01:32.232071


In [283]:
''' Add columns '''

tmp_df = agg_df
tmp_df = tmp_df[tmp_df['METRIC_CD'].str.contains('B0R00010001')]
tmp_df['TMP_CD'] = tmp_df['METRIC_CD'].replace(r'AA$|AB$|AC$|AD$|AE$|AF$|AG$|AH$|AI$|AJ$|AK$', '', regex=True)
tmp_df['TMP_NAME'] = tmp_df['METRIC_NAME'].replace(r' : Account Executive| : B2B| : Branded Retail| : Contact Center| : Direct Sales| : Key Account| : Modern Trade| : Others| : Own Digital| : Retail Sales| : Wholesales', '', regex=True)
tmp_df

C:\Users\Narut4\AppData\Local\Temp\ipykernel_10316\1398469473.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['TMP_CD'] = tmp_df['METRIC_CD'].replace(r'AA$|AB$|AC$|AD$|AE$|AF$|AG$|AH$|AI$|AJ$|AK$', '', regex=True)
C:\Users\Narut4\AppData\Local\Temp\ipykernel_10316\1398469473.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['TMP_NAME'] = tmp_df['METRIC_NAME'].replace(r' : Account Executive| : B2B| : Branded Retail| : Contact Center| : Direct Sales| : Key Account| : Modern Trade| : Others

,COMP_CD,METRIC_CD,METRIC_NAME,CHANNEL_CD,UOM,P_ACTUAL,P_TARGET,LOAD_DATE,TMP_CD,TMP_NAME
1,ALL,B0R00010001,Total Inflow M1,ALL,baht,2.016992e+08,0.000000e+00,2024-06-17 14:01:32.232071,B0R00010001,Total Inflow M1
2,ALL,B0R00010001CORP,Total Inflow M1,ALL,baht,2.016992e+08,0.000000e+00,2024-06-17 14:01:32.232071,B0R00010001CORP,Total Inflow M1
100,DTAC,DB0R00010001,Total Inflow M1 : DTAC,ALL,baht,8.593587e+07,0.000000e+00,2024-06-17 14:01:32.232071,DB0R00010001,Total Inflow M1 : DTAC
188,TRUE,TB0R00010001,Total Inflow M1 : TRUE,ALL,baht,1.157633e+08,1.505547e+08,2024-06-17 14:01:32.232071,TB0R00010001,Total Inflow M1 : TRUE


In [284]:
''' Pivot Table '''

# # Actual & Target
# pv_target = pd.pivot_table(x, values=['P_ACTUAL', 'P_TARGET'], index=['COMP_CD', 'TMP_CD', 'TMP_NAME'], columns='CHANNEL_CD', aggfunc='sum', fill_value=0)

# Actual
pv_actual = pd.pivot_table(tmp_df, values='P_ACTUAL', index=['COMP_CD', 'TMP_CD', 'TMP_NAME'], columns='CHANNEL_CD', aggfunc='sum', fill_value=0)
pv_actual['VERION'] = 'A'

# Target
pv_target = pd.pivot_table(tmp_df, values='P_TARGET', index=['COMP_CD', 'TMP_CD', 'TMP_NAME'], columns='CHANNEL_CD', aggfunc='sum', fill_value=0)
pv_target['VERION'] = 'T'

# Concat Dataframe
pd.concat([pv_actual, pv_target]).reset_index()

CHANNEL_CD,COMP_CD,TMP_CD,TMP_NAME,ALL,VERION
0,ALL,B0R00010001,Total Inflow M1,2.016992e+08,A
1,ALL,B0R00010001CORP,Total Inflow M1,2.016992e+08,A
2,DTAC,DB0R00010001,Total Inflow M1 : DTAC,8.593587e+07,A
3,TRUE,TB0R00010001,Total Inflow M1 : TRUE,1.157633e+08,A
4,ALL,B0R00010001,Total Inflow M1,0.000000e+00,T
5,ALL,B0R00010001CORP,Total Inflow M1,0.000000e+00,T
6,DTAC,DB0R00010001,Total Inflow M1 : DTAC,0.000000e+00,T
7,TRUE,TB0R00010001,Total Inflow M1 : TRUE,1.505547e+08,T


## Generate Output file

In [45]:
# # to Excel file

# op_dir = 'temp'
# op_file = 'VINSIGHT Data Monitoring.xlsx'

# df.to_excel(f'{op_dir}/{op_file}', sheet_name='Data', index=False)
# print(f'\n  -> Generate "{op_file}" successfully')


 -> Generate "Metric_List.xlsx" successfully


In [44]:
# # to CSV file

# op_dir = 'temp'
# op_file = 'VINSIGHT Data Monitoring.csv'

# df.to_csv(f'{op_dir}/{op_file}', index=False, encoding='utf-8')
# print(f'\n  -> Generate "{op_file}" successfully')


 -> Generate "Metric_List.csv" successfully
